In [1]:
from access_nri_intake.source import builders
from dask.distributed import Client
# Hide some warnings
# logging.basicConfig(level=logging.WARNING)

In [2]:
import intake

esm_datastore = intake.open_esm_datastore(
    "/g/data/ps29/nd0349/runs/access-om3/archive/mom6-cice6_iaf_rel/intake_esm_ds.json", 
    columns_with_iterables=[
            "variable",
            "variable_long_name",
            "variable_standard_name",
            "variable_cell_methods",
            "variable_units",
    ] # This is important
)

esm_datastore

,unique
filename,1141
file_id,6
path,1159
filename_timestamp,456
frequency,3
start_date,457
end_date,457
variable,200
variable_long_name,185
variable_standard_name,41


In [3]:
client = Client(threads_per_worker = 1)
client

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41813 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/41813/status,
Dashboard: /proxy/41813/status,Workers: 7
Total threads: 7,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38043,Workers: 7
Dashboard: /proxy/41813/status,Total threads: 7
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:43897,Total threads: 1
Dashboard: /proxy/33105/status,Memory: 4.57 GiB
Nanny: tcp://127.0.0.1:43493,


## Search for unique attributes

In [4]:
esm_datastore.unique().frequency
esm_datastore.search(frequency='1mon').unique().variable
# esm_datastore.unique().variable

['time',
 'time_bounds',
 'NCAT',
 'VGRDi',
 'VGRDs',
 'VGRDb',
 'VGRDa',
 'blkmask',
 'ANGLE',
 'ANGLET',
 'hi_m',
 'hs_m',
 'Tsfc_m',
 'aice_m',
 'uvel_m',
 'vvel_m',
 'uatm_m',
 'vatm_m',
 'sice_m',
 'fswup_m',
 'fswdn_m',
 'flwdn_m',
 'snow_ai_m',
 'rain_ai_m',
 'frzmlt_m',
 'scale_factor_m',
 'fswabs_ai_m',
 'albsni_m',
 'alvdr_ai_m',
 'alidr_ai_m',
 'alvdf_ai_m',
 'alidf_ai_m',
 'albice_m',
 'albsno_m',
 'flat_ai_m',
 'fsens_ai_m',
 'flwup_ai_m',
 'evap_ai_m',
 'Tair_m',
 'congel_m',
 'frazil_m',
 'snoice_m',
 'meltt_m',
 'melts_m',
 'meltb_m',
 'meltl_m',
 'fresh_ai_m',
 'fsalt_m',
 'fsalt_ai_m',
 'fhocn_ai_m',
 'fswthru_ai_m',
 'strairx_m',
 'strairy_m',
 'strtltx_m',
 'strtlty_m',
 'strcorx_m',
 'strcory_m',
 'strocnx_m',
 'strocny_m',
 'strintx_m',
 'strinty_m',
 'strength_m',
 'divu_m',
 'shear_m',
 'vort_m',
 'dvidtt_m',
 'dvidtd_m',
 'daidtt_m',
 'daidtd_m',
 'trsig_m',
 'ice_present_m',
 'fcondtop_ai_m',
 'fmeltt_ai_m',
 'sifb_m',
 'opening_m',
 'aicen_m',
 'vicen_m',
 'f

In [5]:
%%time
# esm_datastore.search(frequency='1mon',variable='aice_m').to_dataset_dict()

# ds = esm_datastore.search(variable="aice_m").search(frequency="1mon").to_dask()
esm_datastore_filtered = esm_datastore.search(
    variable=["aice_m"], frequency="1mon", require_all_on="path"
)

CPU times: user 1.89 s, sys: 132 ms, total: 2.03 s
Wall time: 1.88 s


In [17]:
esm_datastore_filtered

,unique
filename,228
file_id,1
path,228
filename_timestamp,228
frequency,1
start_date,228
end_date,228
variable,82
variable_long_name,82
variable_standard_name,1


In [13]:
%%time
xarray_open_kwargs = {"chunks": {"time": 120, "nj": -1, "ni": -1}}

ds = esm_datastore_filtered.to_dask(xarray_open_kwargs=xarray_open_kwargs)#.aice_m

2025-05-28 11:34:49,696 - distributed.worker - WARNING - Compute Failed
Key:       _open_dataset-ffccf258-f97b-437c-b319-019dca43a6ad
State:     executing
Function:  execute_task
args:      ((<function apply at 0x15373bc16830>, <function _open_dataset at 0x1537190b6050>, ['/g/data/ps29/nd0349/spack/0.22/spack-release/mom6-cice6_configs/mom6-cice6_iaf_rel/archive/output001/access-om3.cice.1mon.mean.1959-06.nc', ['time', 'time_bounds', 'NCAT', 'VGRDi', 'VGRDs', 'VGRDb', 'VGRDa', 'blkmask', 'ANGLE', 'ANGLET', 'hi_m', 'hs_m', 'Tsfc_m', 'aice_m', 'uvel_m', 'vvel_m', 'uatm_m', 'vatm_m', 'sice_m', 'fswup_m', 'fswdn_m', 'flwdn_m', 'snow_ai_m', 'rain_ai_m', 'frzmlt_m', 'scale_factor_m', 'fswabs_ai_m', 'albsni_m', 'alvdr_ai_m', 'alidr_ai_m', 'alvdf_ai_m', 'alidf_ai_m', 'albice_m', 'albsno_m', 'flat_ai_m', 'fsens_ai_m', 'flwup_ai_m', 'evap_ai_m', 'Tair_m', 'congel_m', 'frazil_m', 'snoice_m', 'meltt_m', 'melts_m', 'meltb_m', 'meltl_m', 'fresh_ai_m', 'fsalt_m', 'fsalt_ai_m', 'fhocn_ai_m', 'fswthru_

ESMDataSourceError: Failed to load dataset with key='access_om3_cice_1mon_mean_XXXX_XX.1mon'
                 You can use `cat['access_om3_cice_1mon_mean_XXXX_XX.1mon'].df` to inspect the assets/files for this key.
                 

2025-05-28 11:34:50,068 - distributed.worker - WARNING - Compute Failed
Key:       _open_dataset-cd259f3d-9a44-4454-b6bc-e4d4b10c9cf6
State:     executing
Function:  execute_task
args:      ((<function apply at 0x15373bc16830>, <function _open_dataset at 0x1537190b69e0>, ['/g/data/ps29/nd0349/spack/0.22/spack-release/mom6-cice6_configs/mom6-cice6_iaf_rel/archive/output009/access-om3.cice.1mon.mean.1967-06.nc', ['time', 'time_bounds', 'NCAT', 'VGRDi', 'VGRDs', 'VGRDb', 'VGRDa', 'blkmask', 'ANGLE', 'ANGLET', 'hi_m', 'hs_m', 'Tsfc_m', 'aice_m', 'uvel_m', 'vvel_m', 'uatm_m', 'vatm_m', 'sice_m', 'fswup_m', 'fswdn_m', 'flwdn_m', 'snow_ai_m', 'rain_ai_m', 'frzmlt_m', 'scale_factor_m', 'fswabs_ai_m', 'albsni_m', 'alvdr_ai_m', 'alidr_ai_m', 'alvdf_ai_m', 'alidf_ai_m', 'albice_m', 'albsno_m', 'flat_ai_m', 'fsens_ai_m', 'flwup_ai_m', 'evap_ai_m', 'Tair_m', 'congel_m', 'frazil_m', 'snoice_m', 'meltt_m', 'melts_m', 'meltb_m', 'meltl_m', 'fresh_ai_m', 'fsalt_m', 'fsalt_ai_m', 'fhocn_ai_m', 'fswthru_

In [14]:
ds

2025-05-28 11:34:50,082 - distributed.worker - WARNING - Compute Failed
Key:       _open_dataset-c6bb7e7b-7c86-4968-80ef-b7a70bb67f7b
State:     executing
Function:  execute_task
args:      ((<function apply at 0x152ff22d2830>, <function _open_dataset at 0x152fcb7a6830>, ['/g/data/ps29/nd0349/spack/0.22/spack-release/mom6-cice6_configs/mom6-cice6_iaf_rel/archive/output008/access-om3.cice.1mon.mean.1966-01.nc', ['time', 'time_bounds', 'NCAT', 'VGRDi', 'VGRDs', 'VGRDb', 'VGRDa', 'blkmask', 'ANGLE', 'ANGLET', 'hi_m', 'hs_m', 'Tsfc_m', 'aice_m', 'uvel_m', 'vvel_m', 'uatm_m', 'vatm_m', 'sice_m', 'fswup_m', 'fswdn_m', 'flwdn_m', 'snow_ai_m', 'rain_ai_m', 'frzmlt_m', 'scale_factor_m', 'fswabs_ai_m', 'albsni_m', 'alvdr_ai_m', 'alidr_ai_m', 'alvdf_ai_m', 'alidf_ai_m', 'albice_m', 'albsno_m', 'flat_ai_m', 'fsens_ai_m', 'flwup_ai_m', 'evap_ai_m', 'Tair_m', 'congel_m', 'frazil_m', 'snoice_m', 'meltt_m', 'melts_m', 'meltb_m', 'meltl_m', 'fresh_ai_m', 'fsalt_m', 'fsalt_ai_m', 'fhocn_ai_m', 'fswthru_

NameError: name 'ds' is not defined

n\n    self.f = open(self.path, mode=self.mode)\n'

2025-05-28 11:34:50,083 - distributed.worker - WARNING - Compute Failed
Key:       _open_dataset-c0318565-3d8e-4970-934f-344ac1351f93
State:     executing
Function:  execute_task
args:      ((<function apply at 0x153ddb0f2830>, <function _open_dataset at 0x153db85cb6d0>, ['/g/data/ps29/nd0349/spack/0.22/spack-release/mom6-cice6_configs/mom6-cice6_iaf_rel/archive/output000/access-om3.cice.1mon.mean.1958-01.nc', ['time', 'time_bounds', 'NCAT', 'VGRDi', 'VGRDs', 'VGRDb', 'VGRDa', 'blkmask', 'ANGLE', 'ANGLET', 'hi_m', 'hs_m', 'Tsfc_m', 'aice_m', 'uvel_m', 'vvel_m', 'uatm_m', 'vatm_m', 'sice_m', 'fswup_m', 'fswdn_m', 'flwdn_m', 'snow_ai_m', 'rain_ai_m', 'frzmlt_m', 'scale_factor_m', 'fswabs_ai_m', 'albsni_m', 'alvdr_ai_m', 'alidr_ai_m', 'alvdf_ai_m', 'alidf_ai_m', 'albice_m', 'albsno_m', 'flat_ai_m', 'fsens_ai_m', 'flwup_ai_m', 'evap_ai_m', 'Tair_m', 'congel_m', 'frazil_m', 'snoice_m', 'meltt_m', 'melts_m', 'meltb_m', 'meltl_m', 'fresh_ai_

In [ ]:
# %%time
# ds = esm_datastore.search(variable="aice_m").search(frequency="1mon").to_dask().aice_m
# esm_datastore_filtered.unique().variable
# esm_datastore_filtered
ds["aice_m"].encoding

In [4]:
ds["temp"].isel(time=-1, st_ocean=0).plot()

NameError: name 'ds' is not defined